In [1]:
import sys
import os
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(Path(os.getcwd()).parent))

In [2]:
from loader import Power, Weather
from constant import FeatureType
from keras.layers import LSTM, RepeatVector, TimeDistributed, Dense
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSProp
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error,r2_score
from datetime import datetime, timedelta

import json
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf

### Setting

In [3]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ====== Path ====== #
args.root = root

# ====== Model ====== #
args.frame_in = 72
args.frame_out = 24

# ====== Data ====== #
args.years = [2017, 2018, 2019]
args.region = "Jindo"
args.station = 192
args.ratio = [0.6, 0.2, 0.2]

# ====== Features ====== #
features = [FeatureType.SUNSHINE,
            FeatureType.GROUND_TEMPERATURE,
            FeatureType.HUMIDITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.PRECIPITATION,
            FeatureType.STEAM_PRESSURE,
            FeatureType.DEW_POINT_TEMPERATURE,
            FeatureType.ATMOSPHERIC_PRESSURE]

### Saved settings

In [4]:
result_path = os.path.join(args.root, 'results', 'automl_optimal_settings.json')
with open(result_path) as f:
    data = json.load(f)

settings = []
for setting in data['results']:
    settings.append(setting)

### Model

In [5]:
def encoder_decoder(args, dataset, budget):
    print("train")

    X_train, y_train = dataset['train']
    X_val, y_val = dataset['val']
    X_test, y_test = dataset['test']

    with tf.device('/GPU:0'):
        model = Sequential()
        optimizer = RMSProp(learning_rate=args.learning_rate)

        model.add(LSTM(256, input_shape=(args.frame_in, args.feature_len)))
        model.add(RepeatVector(args.frame_out))
        model.add(LSTM(256, return_sequences=True))
        model.add(TimeDistributed(Dense(256, activation='relu')))
        model.add(TimeDistributed(Dense(1)))
        model.compile(loss='mse', optimizer=optimizer)

        print("model name: %s" % args.name)
        
        model_root_path = os.path.join(root, 'models', args.name)
        Path(model_root_path).mkdir(parents=True, exist_ok=True)
        checkpoint_path = os.path.join(model_root_path, 'model-{epoch:03d}-{val_loss:03f}.h5')
        model_path = os.path.join(model_root_path, 'model.h5')
        
        callback = EarlyStopping(monitor='val_loss', patience=args.patience)
        checkpoint = ModelCheckpoint(checkpoint_path, verbose=1, monitor='val_loss', save_best_only=True)

        history = model.fit(X_train, y_train, batch_size=args.batch_size, epochs=budget,
                            validation_data=(X_val, y_val), callbacks=[callback, checkpoint], shuffle=args.shuffle)
        model.save(model_path)
        
        return model
    
def predict(partition, model, scaler):
    X_test, y_test = partition['test']
    y_test = y_test.reshape((y_test.shape[0] * y_test.shape[1], y_test.shape[2]))
    y_test = scaler.inverse_transform(y_test)
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape((-1, 1))
    y_pred = scaler.inverse_transform(y_pred)
    
    print(y_test.shape, y_pred.shape)
    
    zero_indices = np.where(y_test == 0)
    y_test_adjusted = np.delete(y_test, zero_indices)
    y_pred_adjusted = np.delete(y_pred, zero_indices)

    rmse = np.sqrt(mean_squared_error(y_test_adjusted, y_pred_adjusted))
    max_min = np.max(y_test_adjusted) - np.min(y_test_adjusted)
    nrmse = rmse / max_min
    
    print('rmse:', rmse)
    print('nrmse:', nrmse)
    
    return y_test, y_pred, nrmse

def save(y_pred_list, y_test, test_start, test_end, name='result.csv'):
    df = pd.DataFrame()
    for i in range(len(features)):
        y_pred_list[i] = y_pred_list[i].reshape((y_pred_list[i].shape[0]))
        df['%dth model' % (i + 1)] = y_pred_list[i].tolist()
    y_test = y_test.reshape((y_test.shape[0]))
    df['y_test'] = y_test.tolist()

    full_idx = pd.date_range(start=test_start+timedelta(days=3), end=test_end, freq='H')
    full_idx = full_idx[:y_test.shape[0]]
    df['time'] = full_idx
    df = df.set_index('time')

    result_path = os.path.join(root, 'results')
    result_name = os.path.join(result_path, name)
    df.to_csv(result_name)

### Experiment

In [6]:
power = Power(args)
weather = Weather(args, features)

power_data = power.get_data()
scaler = power_data['scaler']

y_pred_list = []
nrmse_list = []

for i in range(len(features)):
    weather_data = weather.get_data(i+1)
    setattr(args, 'feature_len', i + 1)
    setattr(args, 'name', 'fixed_learning_rate_model_%d' % (i + 1))
    setattr(args, 'batch_size', settings[i]['batch_size'])
    setattr(args, 'learning_rate', 0.001)
    setattr(args, 'patience', settings[i]['patience'])
    setattr(args, 'shuffle', True if settings[i]['shuffle'] == 'True' else False)

    train = [weather_data['train'], power_data['train']]
    val = [weather_data['val'], power_data['val']]
    test = [weather_data['test'], power_data['test']]

    dataset = {'train': train, 'val': val, 'test': test}
    
    model = encoder_decoder(args, dataset, budget=256)
    y_test, y_pred, nrmse = predict(dataset, model, scaler)
    y_pred_list.append(y_pred)
    nrmse_list.append(nrmse)
    
save(y_pred_list, y_test, power.test_start, power.test_end, name='fix_learning_rate.csv')

train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
train start date: 2017-01-01 00:00:00
train end date: 2018-10-19 23:00:00
val start date: 2018-10-20 00:00:00
val end date: 2019-05-26 23:00:00
test start date: 2019-05-27 00:00:00
test end date: 2019-12-31 23:00:00
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
0 missing dates
0 value(s) are not zero
(15768,)
(5256,)
(5256,)
missing dates: []
missing dates: ['2018-01-13 09:00', '2018-01-13 10:00', '2018-01-13 11:00', '2018-01-13 12:00', '2018-01-13 13:00']
missing dates: []
train
model name: fixed_learning_rate_model_1
Epoch 1/256
5/6 [========================>.....] - ETA: 0s - loss: 0.0052
Epoch 00001: val_loss improved from inf to 0.00147, saving model to /home/vislab/Github/SolarPowerPrediction/models/fixed_learning_rate_model_1/model-0

6/6 [==============================] - ETA: 0s - loss: 0.0016
Epoch 00008: val_loss did not improve from 0.00138
6/6 [==============================] - 0s 24ms/step - loss: 0.0016 - val_loss: 0.0018
Epoch 9/256
5/6 [========================>.....] - ETA: 0s - loss: 0.0018
Epoch 00009: val_loss improved from 0.00138 to 0.00130, saving model to /home/vislab/Github/SolarPowerPrediction/models/fixed_learning_rate_model_2/model-009-0.001300.h5
6/6 [==============================] - 0s 51ms/step - loss: 0.0018 - val_loss: 0.0013
Epoch 10/256
5/6 [========================>.....] - ETA: 0s - loss: 0.0014
Epoch 00010: val_loss did not improve from 0.00130
6/6 [==============================] - 0s 15ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 11/256
5/6 [========================>.....] - ETA: 0s - loss: 0.0015
Epoch 00011: val_loss did not improve from 0.00130
6/6 [==============================] - 0s 27ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 12/256
4/6 [===================>.........

Epoch 41/256
4/6 [===================>..........] - ETA: 0s - loss: 4.3314e-04
Epoch 00041: val_loss did not improve from 0.00032
6/6 [==============================] - 0s 20ms/step - loss: 5.1281e-04 - val_loss: 4.9140e-04
Epoch 42/256
5/6 [========================>.....] - ETA: 0s - loss: 5.9397e-04
Epoch 00042: val_loss did not improve from 0.00032
6/6 [==============================] - 0s 48ms/step - loss: 5.8813e-04 - val_loss: 4.1182e-04
Epoch 43/256
5/6 [========================>.....] - ETA: 0s - loss: 5.6041e-04
Epoch 00043: val_loss did not improve from 0.00032
6/6 [==============================] - 0s 15ms/step - loss: 5.5464e-04 - val_loss: 3.5745e-04
Epoch 44/256
5/6 [========================>.....] - ETA: 0s - loss: 5.0697e-04
Epoch 00044: val_loss did not improve from 0.00032
6/6 [==============================] - 0s 24ms/step - loss: 5.0358e-04 - val_loss: 4.3623e-04
Epoch 45/256
6/6 [==============================] - ETA: 0s - loss: 5.8407e-04
Epoch 00045: val_loss did

Epoch 26/256
322/327 [============================>.] - ETA: 0s - loss: 3.9262e-04
Epoch 00026: val_loss did not improve from 0.00036
327/327 [==============================] - 3s 9ms/step - loss: 3.9030e-04 - val_loss: 3.9169e-04
Epoch 27/256
327/327 [==============================] - ETA: 0s - loss: 3.9152e-04
Epoch 00027: val_loss did not improve from 0.00036
327/327 [==============================] - 4s 11ms/step - loss: 3.9152e-04 - val_loss: 3.9657e-04
Epoch 28/256
324/327 [============================>.] - ETA: 0s - loss: 3.9253e-04
Epoch 00028: val_loss did not improve from 0.00036
327/327 [==============================] - 4s 11ms/step - loss: 3.9135e-04 - val_loss: 4.1257e-04
Epoch 29/256
317/327 [============================>.] - ETA: 0s - loss: 3.9180e-04
Epoch 00029: val_loss did not improve from 0.00036
327/327 [==============================] - 3s 10ms/step - loss: 3.8972e-04 - val_loss: 4.1371e-04
Epoch 30/256
326/327 [============================>.] - ETA: 0s - loss: 3

Epoch 25/256
324/327 [============================>.] - ETA: 0s - loss: 3.8970e-04
Epoch 00025: val_loss did not improve from 0.00035
327/327 [==============================] - 3s 10ms/step - loss: 3.8875e-04 - val_loss: 3.5744e-04
Epoch 26/256
320/327 [============================>.] - ETA: 0s - loss: 3.8674e-04
Epoch 00026: val_loss did not improve from 0.00035
327/327 [==============================] - 4s 11ms/step - loss: 3.8505e-04 - val_loss: 3.5269e-04
Epoch 27/256
317/327 [============================>.] - ETA: 0s - loss: 3.8540e-04
Epoch 00027: val_loss did not improve from 0.00035
327/327 [==============================] - 3s 9ms/step - loss: 3.8445e-04 - val_loss: 3.5246e-04
Epoch 28/256
324/327 [============================>.] - ETA: 0s - loss: 3.8254e-04
Epoch 00028: val_loss did not improve from 0.00035
327/327 [==============================] - 3s 11ms/step - loss: 3.8166e-04 - val_loss: 3.5391e-04
Epoch 29/256
324/327 [============================>.] - ETA: 0s - loss: 3

...

Epoch 75/256
6/6 [==============================] - ETA: 0s - loss: 5.4348e-04
Epoch 00075: val_loss did not improve from 0.00029
6/6 [==============================] - 0s 13ms/step - loss: 5.4348e-04 - val_loss: 4.0839e-04
Epoch 76/256
1/6 [====>.........................] - ETA: 0s - loss: 4.6266e-04
Epoch 00076: val_loss did not improve from 0.00029
6/6 [==============================] - 0s 13ms/step - loss: 4.7531e-04 - val_loss: 2.9484e-04
Epoch 77/256
1/6 [====>.........................] - ETA: 0s - loss: 3.3615e-04
Epoch 00077: val_loss did not improve from 0.00029
6/6 [==============================] - 0s 13ms/step - loss: 4.0912e-04 - val_loss: 4.3041e-04
Epoch 78/256
1/6 [====>.........................] - ETA: 0s - loss: 3.7852e-04
Epoch 00078: val_loss did not improve from 0.00029
6/6 [==============================] - 0s 14ms/step - loss: 4.5553e-04 - val_loss: 3.9349e-04
Epoch 79/256
1/6 [====>.........................] - ETA: 0s - loss: 7.1020e-04
Epoch 00079: val_loss did

Epoch 111/256
6/6 [==============================] - ETA: 0s - loss: 4.0061e-04
Epoch 00111: val_loss did not improve from 0.00026
6/6 [==============================] - 0s 13ms/step - loss: 4.0061e-04 - val_loss: 3.0102e-04
Epoch 112/256
1/6 [====>.........................] - ETA: 0s - loss: 6.4859e-04
Epoch 00112: val_loss did not improve from 0.00026
6/6 [==============================] - 0s 13ms/step - loss: 3.7718e-04 - val_loss: 5.2572e-04
Epoch 113/256
1/6 [====>.........................] - ETA: 0s - loss: 5.7546e-04
Epoch 00113: val_loss did not improve from 0.00026
6/6 [==============================] - 0s 13ms/step - loss: 5.1624e-04 - val_loss: 3.9363e-04
Epoch 114/256
1/6 [====>.........................] - ETA: 0s - loss: 7.2674e-04
Epoch 00114: val_loss did not improve from 0.00026
6/6 [==============================] - 0s 13ms/step - loss: 3.8573e-04 - val_loss: 2.9580e-04
Epoch 115/256
1/6 [====>.........................] - ETA: 0s - loss: 3.2285e-04
Epoch 00115: val_los

Epoch 26/256
326/327 [============================>.] - ETA: 0s - loss: 3.7381e-04
Epoch 00026: val_loss did not improve from 0.00026
327/327 [==============================] - 2s 5ms/step - loss: 3.7311e-04 - val_loss: 2.8926e-04
Epoch 27/256
325/327 [============================>.] - ETA: 0s - loss: 3.7819e-04
Epoch 00027: val_loss did not improve from 0.00026
327/327 [==============================] - 2s 5ms/step - loss: 3.7720e-04 - val_loss: 3.0225e-04
Epoch 28/256
326/327 [============================>.] - ETA: 0s - loss: 3.6886e-04
Epoch 00028: val_loss did not improve from 0.00026
327/327 [==============================] - 2s 5ms/step - loss: 3.6788e-04 - val_loss: 2.6352e-04
Epoch 29/256
326/327 [============================>.] - ETA: 0s - loss: 3.7328e-04
Epoch 00029: val_loss did not improve from 0.00026
327/327 [==============================] - 2s 5ms/step - loss: 3.7255e-04 - val_loss: 2.9683e-04
Epoch 30/256
323/327 [============================>.] - ETA: 0s - loss: 3.70

34/41 [=======================>......] - ETA: 0s - loss: 4.2312e-04
Epoch 00021: val_loss improved from 0.00028 to 0.00027, saving model to /home/vislab/Github/SolarPowerPrediction/models/fixed_learning_rate_model_11/model-021-0.000271.h5
41/41 [==============================] - 0s 7ms/step - loss: 4.1540e-04 - val_loss: 2.7149e-04
Epoch 22/256
34/41 [=======================>......] - ETA: 0s - loss: 4.2698e-04
Epoch 00022: val_loss did not improve from 0.00027
41/41 [==============================] - 0s 6ms/step - loss: 4.1423e-04 - val_loss: 3.1284e-04
Epoch 23/256
34/41 [=======================>......] - ETA: 0s - loss: 4.2887e-04
Epoch 00023: val_loss did not improve from 0.00027
41/41 [==============================] - 0s 6ms/step - loss: 4.1579e-04 - val_loss: 3.2108e-04
Epoch 24/256
34/41 [=======================>......] - ETA: 0s - loss: 3.3021e-04
Epoch 00024: val_loss did not improve from 0.00027
41/41 [==============================] - 0s 6ms/step - loss: 3.9010e-04 - val_lo

34/41 [=======================>......] - ETA: 0s - loss: 3.5149e-04
Epoch 00056: val_loss did not improve from 0.00027
41/41 [==============================] - 0s 6ms/step - loss: 3.5319e-04 - val_loss: 3.1028e-04
Epoch 57/256
34/41 [=======================>......] - ETA: 0s - loss: 3.6238e-04
Epoch 00057: val_loss did not improve from 0.00027
41/41 [==============================] - 0s 6ms/step - loss: 3.5784e-04 - val_loss: 2.9541e-04
(5184, 1) (5184, 1)
rmse: 46.163779787842024
nrmse: 0.15037061820143982


In [7]:
nrmse_list

[0.1951336258449734,
 0.19979368632598277,
 0.18124959690359377,
 0.1771501202916675,
 0.18375330622369798,
 0.1594900000767185,
 0.15971574002409786,
 0.18310370826068534,
 0.1568348828820687,
 0.16202194039253437,
 0.15037061820143982]